In [1]:
import matplotlib as mpl

# This line allows mpl to run with no DISPLAY defined
mpl.use('Agg')

import pandas as pd
import numpy as np
import os
from keras.layers import Reshape, Flatten, LeakyReLU, Activation
from keras.layers.convolutional import UpSampling2D, MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras_adversarial.image_grid_callback import ImageGridCallback

from keras_adversarial import AdversarialModel, simple_gan, gan_targets
from keras_adversarial import AdversarialOptimizerSimultaneous, normal_latent_sampling
from keras_adversarial.legacy import Dense, BatchNormalization, fit, l1l2, Convolution2D, AveragePooling2D
from keras.layers import Conv2D
import keras.backend as K
from keras.datasets import cifar10
#from cifar10_utils import cifar10_data
#from image_utils import dim_ordering_fix, dim_ordering_unfix, dim_ordering_shape

import numpy as np
import matplotlib.pyplot as plt
import scipy.misc
import os
import imageio

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [36]:
class GAN(object):
    def __init__(self, width = 32, height= 32, channels = 3):

        self.WIDTH = width
        self.HEIGHT = height
        self.CHANNELS = channels

        self.SHAPE = (self.WIDTH, self.HEIGHT, self.CHANNELS)

        self.OPTIMIZER = Adam(lr=0.0002, decay=8e-9)

        self.noise_gen = np.random.normal(0,1,(100,))

        self.G = self.generator()
        self.G.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER)

        self.D = self.discriminator()
        self.D.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER, metrics=['accuracy'] )

        self.stacked_G_D = self.stacked_G_D()

        self.stacked_G_D.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER)

    def generator(self):
        model = Sequential()
        nch = 256
        reg = lambda: l1l2(l1=1e-7, l2=1e-7)
        h = 5
        model.add(Dense(nch * 4 * 4, input_dim=100))
        model.add(BatchNormalization(mode=0))
        model.add(Reshape((4, 4, nch)))
        model.add(Conv2D(int(nch / 2), (h, h), border_mode='same'))
        model.add(BatchNormalization(mode=0, axis=1))
        model.add(LeakyReLU(0.2))
        model.add(UpSampling2D(size=(2, 2)))
        model.add(Conv2D(int(nch / 2), (h, h), border_mode='same'))
        model.add(BatchNormalization(mode=0, axis=1))
        model.add(LeakyReLU(0.2))
        model.add(UpSampling2D(size=(2, 2)))
        model.add(Conv2D(int(nch / 4), (h, h), border_mode='same'))
        model.add(BatchNormalization(mode=0, axis=1))
        model.add(LeakyReLU(0.2))
        model.add(UpSampling2D(size=(2, 2)))
        model.add(Conv2D(3, (h, h), border_mode='same'))
        model.add(Activation('sigmoid'))
        model.summary()
        return model
    
    
    def discriminator(self):
        nch = 256
        h = 5
        reg = lambda: l1l2(l1=1e-7, l2=1e-7)

        c1 = Conv2D(h, h, int(nch / 4), border_mode='same', W_regularizer=reg(),
                           input_shape=self.SHAPE)
        c2 = Conv2D(int(nch / 2), (h, h), border_mode='same', W_regularizer=reg())
        c3 = Conv2D(nch, (h, h), border_mode='same', W_regularizer=reg())
        c4 = Conv2D(1, (h, h), border_mode='same', W_regularizer=reg())

        model = Sequential()
        model.add(c1)
        model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
        model.add(LeakyReLU(0.2))
        model.add(c2)
        model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
        model.add(LeakyReLU(0.2))
        model.add(c3)
        model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
        model.add(LeakyReLU(0.2))
        model.add(c4)
        model.add(AveragePooling2D(pool_size=(2, 2), border_mode='valid'))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        return model

    def stacked_G_D(self):
        self.D.trainable = False

        model = Sequential()
        model.add(self.G)
        model.add(self.D)

        return model

    def train(self, X_train, epochs=20000, batch = 512, save_interval = 100):

        d_loss = 500
        g_loss = 500
        for cnt in range(30000):

            if cnt % 10 == 0:
                for ix in range(10):
                    ## train discriminator
                    random_index =  np.random.randint(0, len(X_train) - int(batch/2))
                    legit_images = X_train[random_index : random_index + int(batch/2)]
                    legit_images = legit_images.reshape(int(batch/2), self.WIDTH, self.HEIGHT, self.CHANNELS)

                    gen_noise = np.random.normal(0, 1, (int(batch/2),100))
                    syntetic_images = self.G.predict(gen_noise)

                    x_combined_batch = np.concatenate((legit_images, syntetic_images))
                    y_combined_batch = np.concatenate((np.ones((int(batch/2), 1)), np.zeros((int(batch/2), 1))))

                    d_loss = self.D.train_on_batch(x_combined_batch, y_combined_batch)
                    print(ix, end='')
                print()

                    
            else:
                # train generator
                noise = np.random.normal(0, 1, (batch,100))
                y_mislabled = np.ones((batch, 1))

                g_loss = self.stacked_G_D.train_on_batch(noise, y_mislabled)
                print('.', end='')

            if cnt % 1 == 0:
                print ('epoch: %d, [Discriminator :: d_loss: %f] [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

            if cnt % 100 == 0 : 
                self.save_images(cnt=cnt)

    def save_images(self, cnt, samples=10):
        noise = np.random.normal(0, 1, (samples,100))

        images = self.G.predict(noise)
        for ix, image in enumerate(images):
#             image = image.reshape(32, 32)
            if not os.path.exists('results/{:05d}'.format(cnt)):
                os.makedirs('results/{:05d}'.format(cnt))
            imageio.imwrite('results/{:05d}/{}.jpg'.format(cnt, ix), image)

In [37]:
(X_train, y_train), (_, _) = cifar10.load_data()
X_train = X_train[np.where(y_train == 3)[0]]
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

# X_train = np.zeros((50000, 32, 32, 3))
# X_train[:, 10:22, 10:22] = 1

In [ ]:
gan = GAN()
gan.train(X_train)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), padding="same")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), padding="same")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (5, 5), padding="same")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (5, 64), input_shape=(32, 32, 3..., padding="same", kernel_regularizer=<keras.reg...)`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 4096)              413696    
_________________________________________________________________
batch_normalization_49 (Batc (None, 4096)              16384     
_________________________________________________________________
reshape_13 (Reshape)         (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 4, 4, 128)         819328    
_________________________________________________________________
batch_normalization_50 (Batc (None, 4, 4, 128)         16        
_________________________________________________________________
leaky_re_lu_73 (LeakyReLU)   (None, 4, 4, 128)         0         
_________________________________________________________________
up_sampling2d_37 (UpSampling (None, 8, 8, 128)         0         
__________

/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


012345678

9
epoch: 0, [Discriminator :: d_loss: 0.449916] [ Generator :: loss: 500.000000]
.epoch: 1, [Discriminator :: d_loss: 0.449916] [ Generator :: loss: 1.400891]
.epoch: 2, [Discriminator :: d_loss: 0.449916] [ Generator :: loss: 1.099286]
.epoch: 3, [Discriminator :: d_loss: 0.449916] [ Generator :: loss: 0.845396]
.epoch: 4, [Discriminator :: d_loss: 0.449916] [ Generator :: loss: 0.706008]
.epoch: 5, [Discriminator :: d_loss: 0.449916] [ Generator :: loss: 0.646789]
.epoch: 6, [Discriminator :: d_loss: 0.449916] [ Generator :: loss: 0.615699]
.epoch: 7, [Discriminator :: d_loss: 0.449916] [ Generator :: loss: 0.584927]
.epoch: 8, [Discriminator :: d_loss: 0.449916] [ Generator :: loss: 0.547819]
.epoch: 9, [Discriminator :: d_loss: 0.449916] [ Generator :: loss: 0.502830]
0123456789
epoch: 10, [Discriminator :: d_loss: 0.411552] [ Generator :: loss: 0.502830]
.epoch: 11, [Discriminator :: d_loss: 0.411552] [ Generator :: loss: 0.902813]
.epoch: 12, [Discriminator :: d_loss: 0.411552] [

...

9
epoch: 1300, [Discriminator :: d_loss: 0.040806] [ Generator :: loss: 4.289055]
.epoch: 1301, [Discriminator :: d_loss: 0.040806] [ Generator :: loss: 4.287536]
.epoch: 1302, [Discriminator :: d_loss: 0.040806] [ Generator :: loss: 4.228196]
.epoch: 1303, [Discriminator :: d_loss: 0.040806] [ Generator :: loss: 4.125185]
.epoch: 1304, [Discriminator :: d_loss: 0.040806] [ Generator :: loss: 4.050664]
.epoch: 1305, [Discriminator :: d_loss: 0.040806] [ Generator :: loss: 4.006654]
.epoch: 1306, [Discriminator :: d_loss: 0.040806] [ Generator :: loss: 3.895222]
.epoch: 1307, [Discriminator :: d_loss: 0.040806] [ Generator :: loss: 3.834297]
.epoch: 1308, [Discriminator :: d_loss: 0.040806] [ Generator :: loss: 3.778961]
.epoch: 1309, [Discriminator :: d_loss: 0.040806] [ Generator :: loss: 3.728902]
0123456789
epoch: 1310, [Discriminator :: d_loss: 0.029071] [ Generator :: loss: 3.728902]
.epoch: 1311, [Discriminator :: d_loss: 0.029071] [ Generator :: loss: 5.431823]
.epoch: 1312, [Di

012345678

9
epoch: 1400, [Discriminator :: d_loss: 0.019296] [ Generator :: loss: 4.722061]
.epoch: 1401, [Discriminator :: d_loss: 0.019296] [ Generator :: loss: 4.501178]
.epoch: 1402, [Discriminator :: d_loss: 0.019296] [ Generator :: loss: 4.452823]
.epoch: 1403, [Discriminator :: d_loss: 0.019296] [ Generator :: loss: 4.396958]
.epoch: 1404, [Discriminator :: d_loss: 0.019296] [ Generator :: loss: 4.341307]
.epoch: 1405, [Discriminator :: d_loss: 0.019296] [ Generator :: loss: 4.259142]
.epoch: 1406, [Discriminator :: d_loss: 0.019296] [ Generator :: loss: 4.233591]
.epoch: 1407, [Discriminator :: d_loss: 0.019296] [ Generator :: loss: 4.090741]
.epoch: 1408, [Discriminator :: d_loss: 0.019296] [ Generator :: loss: 4.059753]
.epoch: 1409, [Discriminator :: d_loss: 0.019296] [ Generator :: loss: 3.907192]
0123456789
epoch: 1410, [Discriminator :: d_loss: 0.022530] [ Generator :: loss: 3.907192]
.epoch: 1411, [Discriminator :: d_loss: 0.022530] [ Generator :: loss: 4.728812]
.epoch: 1412, [Di

012345678

9
epoch: 1500, [Discriminator :: d_loss: 0.030753] [ Generator :: loss: 3.830735]
.epoch: 1501, [Discriminator :: d_loss: 0.030753] [ Generator :: loss: 5.390893]
.epoch: 1502, [Discriminator :: d_loss: 0.030753] [ Generator :: loss: 5.272077]
.epoch: 1503, [Discriminator :: d_loss: 0.030753] [ Generator :: loss: 5.183721]
.epoch: 1504, [Discriminator :: d_loss: 0.030753] [ Generator :: loss: 4.934861]
.epoch: 1505, [Discriminator :: d_loss: 0.030753] [ Generator :: loss: 4.846025]
.epoch: 1506, [Discriminator :: d_loss: 0.030753] [ Generator :: loss: 4.708374]
.epoch: 1507, [Discriminator :: d_loss: 0.030753] [ Generator :: loss: 4.514067]
.epoch: 1508, [Discriminator :: d_loss: 0.030753] [ Generator :: loss: 4.427891]
.epoch: 1509, [Discriminator :: d_loss: 0.030753] [ Generator :: loss: 4.203860]
0123456789
epoch: 1510, [Discriminator :: d_loss: 0.002534] [ Generator :: loss: 4.203860]
.epoch: 1511, [Discriminator :: d_loss: 0.002534] [ Generator :: loss: 5.580693]
.epoch: 1512, [Di